In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

### Why do we use Python?

+ Readability and simplicity
+ Rapid development and protyping
+ Rich ecosystem of libraries and frameworks
+ Large and helpful community

### What are we missing?

+ Speed?

## Numba

 You have some code... you want to make it run fast 

## How Numba works

In [ ]:
def integrate(a, b, n):
    s = 0.0
    dx = (b - a) / n
    for i in range(n):
        x = a + (i + 0.5) * dx
        y = x ** 4 - 3 * x
        s += y * dx
    return s

In [ ]:
a = 0
b = 2
n = 1_000_000

In [ ]:
# Measure execution time

plaintime = %timeit -o integrate(a, b, n)

In [ ]:
#Add Numba decorator here


In [ ]:
# First thing: test correctness

np.testing.assert_almost_equal(integrate(a, b, n), integrate_numba(a, b, n))

In [ ]:
# Redo definition



In [ ]:
# Measure execution time

jittedtime = %timeit -o -r 1 -n 1 integrate_numba(a, b, n)

In [ ]:
plaintime.best / jittedtime.best

In [ ]:
# Test timing again

jittedtime = %timeit -o integrate_numba(a, b, n)

In [ ]:
plaintime.best / jittedtime.best

<font size = 6>Why is there a slow run? </font>

Numba has to compile your function for the argument types given before it executes the machine code version of your function. This takes time. 

However, once the compilation has taken place Numba caches the machine code version of your function for the particular types of datatypes presented. 

If it is called again, it can reuse the cached version, if it sees the same datatypes, instead of having to compile again.

In [ ]:
integrate

In [ ]:
integrate_numba

In [ ]:
# Define function again to start with a clean slate

@numba.njit
def numba_integrate(a, b, n):
    s = 0.0
    dx = (b - a) / n
    for i in range(n):
        x = a + (i + 0.5) * dx
        y = x ** 4 - 3 * x
        s += y * dx
    return s 


In [ ]:
numba_integrate.signatures

In [ ]:
numba_integrate(0, 2, 1_000_000)

In [ ]:
numba_integrate.signatures

In [ ]:
numba_integrate(0.0j, 2.0j, 1_000_000.0)

In [ ]:
numba_integrate.signatures

In [ ]:
@numba.njit
def numba_integrate(a, b, n):
    s = 0.0
    dx = (b - a) / n
    for i in range(n):
        x = a + (i + 0.5) * dx
        y = x ** 4 - 3 * x
        s += y * dx
    return s          

# COMPILATION TIME!
start = time.perf_counter()
numba_integrate(a, b, n)
end = time.perf_counter()
print(f"Elapsed (with compilation) = {end - start:.4f} s")

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
for i in range(5):
    start = time.perf_counter()
    numba_integrate(a, b, n)
    end = time.perf_counter()
    print(f"Elapsed (cached)           = {end - start:.4f} s")

# Original function
start = time.perf_counter()
numba_integrate.py_func(a, b, n)
end = time.perf_counter()
print(f"Elapsed (python code)      = {end - start:.4f} s")

<font size=8> **Hands on exercise** <font>

Function ```estimate_pi``` calculates the value of $pi$

Notice that the greater the value of n, the better is the estimate of $pi$

You task is to:
+ Use time or timeit to benchmark the speed up in execution of the function for different *n* with and without numba
+ Plot the difference in time for different *n* with and without numba
+ Take a photo of the screen with the plot and post it to the telegram group

**(Ensure that the numba function is compiled before timing the function)**

In [ ]:
# This function takes n as input which is the number of total points within the square
# It then estimates how many of those points are within the circle
# value of pi = 4 * numberofpointsincircle/numberofpointsinsquare

def estimate_pi(n):
    np.random.seed(1234)
    count_inside_circle = 0
    for _ in range(n): # Numba likes loops and numpy functions
        x, y  = [np.random.uniform(-1, 1),
                 np.random.uniform(-1, 1)]
        
        if x**2 + y**2 <= 1:
            count_inside_circle += 1
    
    pi_estimate = 4 * count_inside_circle / n #Numba likes math
    return pi_estimate

In [ ]:
n = [10, 1000, 10000, 100_000, 1_000_000]
for i in n:
    pi_estimate = estimate_pi(i)
    print(f'Number of runs {i}, Estimated \N{GREEK CAPITAL LETTER PI}: {pi_estimate:0.2f}')

In [ ]:
%timeit estimate_pi(1_000_000)

In [ ]:
## Your code here

In [ ]:
n = [10, 1000, 10000, 100_000, 1_000_000]
for i in n:
    pi_estimate = estimate_pi(i)
    print(f'Number of runs {i}, Estimated \N{GREEK CAPITAL LETTER PI}: {pi_estimate:0.2f}')

In [ ]:
%timeit estimate_pi(1_000_000)

In [ ]:
# One possible solution: %load 0-numba-pi/solution.py

## What can you use numba on?

If your code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice.

* Functions with loops
* Functions with arithmetic operations
* Functions with logical operations
* Functions using numpy
* njit-ed functions calling other njit-ed funtions

***OK***: tuples, strings

***Not good***: objects, Python lists, python dicts

> @njit stands for @jit(nopython=True) and it means that numba should throw an exception instead of falling back to the python version, in cases where it cannot generate the optimized version.

## What does numba *not* do? : Choose wisely!
* Magically make individual numpy functions faster, or speed up code that just has nice array operations
* Convert arbitrary python data types
* Translate entire programs
* Compile third party libraries
* Numba cannot help speed up programs that are not CPU-bound

## Compiled code vs Interpreted Code

![](figures/program_life_python.png)

![](figures/program_life_compiler.png)

![](figures/program_life.png)

![](figures/program_with_dll.png)